In [3172]:
import json
import altair as alt
from altair import expr, datum
import altair_viewer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [3173]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3174]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [3175]:
%%capture pwd
!pwd

In [3176]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = (
    eco_git_home + "magazine/" + uid + "/data/"
)  #!!! make sure this points to the magazine
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('issue-2', 300, 500)

# Article 4
## Climate in numbers

In [3177]:
readme = "# Article 4 \n\n## Climate in numbers\n\n"
open("README.md", "w").write(readme)

37

### Fig 4-1

In [3178]:
configSource = "raw/ECO2.DATA.C1.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3179]:
f = "fig4-1_fossil-fuels"
f4_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Time,Value,Source
0,1850,569,Coal
1,1860,1061,Coal
2,1870,1642,Coal
3,1880,2542,Coal
4,1890,3856,Coal


In [3180]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_1
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["title"]["dy"] = -5
config["mark"] = {"type": "area", "opacity": 0.9}
config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config)
text = (
    base.mark_text(dy=35, color=colors["eco-gray"])
    .encode(
        text="Source:N",
        y=alt.Y("Value:Q", stack=True),
        color=alt.Color("c:N", scale=None),
    )
    .transform_calculate(c="'" + scale_lightness(colors["eco-gray"], 0.1) + "'")
    .transform_filter("year(datum.Time)==2012")
)
layer1 = (base + text).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [3181]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 4-2

#### a

In [3182]:
configSource = "raw/ECO2.DATA.C2.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3183]:
f = "fig4-2a_emissions"
f4_2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_2a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_2a = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Time,Country,Value,Value2
0,1901,Brazil,0.113602,National
1,1902,Brazil,0.132434,National
2,1903,Brazil,0.126165,National
3,1904,Brazil,0.132560,National
4,1905,Brazil,0.138663,National


In [3184]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_2a
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["title"]["dy"] = -5
config["encoding"].pop("facet")
config.pop("$schema")
config.pop("resolve")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config)
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country!='Brazil'")
)
text2 = (
    base.mark_text(dx=5, dy=-10, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country=='Brazil'")
)
layer1 = (
    (base + text + text2)
    .transform_filter("datum.Country!='World'")
    .configure_view(stroke=None)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [3185]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (
    (base + text + text2)
    .transform_filter("datum.Country!='World'")
    .configure_view(stroke=None)
)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

#### b

In [3186]:
f = "fig4-2b_emissions"
f4_2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_2b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_2b = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Time,Country,Value,Value2
0,1901,Brazil,0.113602,National
1,1902,Brazil,0.132434,National
2,1903,Brazil,0.126165,National
3,1904,Brazil,0.132560,National
4,1905,Brazil,0.138663,National


In [3187]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_2a
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["title"]["dy"] = -5
config["encoding"].pop("facet")
config.pop("$schema")
config.pop("resolve")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config)
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
)
layer1 = (
    (base + text).transform_filter("datum.Country=='World'").configure_view(stroke=None)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3188]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (
    (base + text).transform_filter("datum.Country=='World'").configure_view(stroke=None)
)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 4-3

In [3189]:
# https://climateactiontracker.org/global/temperatures/
df = pd.read_excel(
    "raw/CAT_2021-05_PublicData_EmissionPathways.xlsx", skiprows=14, nrows=12
).dropna(axis=1, how="all")
df = df.set_index(["Unnamed: 1", "GtCO2e"]).T.reset_index()
df.columns = [
    "year",
    "current_high",
    "current_low",
    "pledges_high",
    "pledges_low",
    "optimistic",
    "c2_high",
    "c2_median",
    "c2_low",
    "c1_high",
    "c1_median",
    "c1_low",
    "historical",
]
df = df.set_index("year").stack().reset_index()
df["scenario"] = df["level_1"].str.split("_").str[0]
df["projection"] = df["level_1"].str.split("_").str[-1]
df = df.set_index(["year", "scenario", "projection"])[0].unstack().reset_index()

/opt/conda/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [3190]:
f = "fig4-3_climate-scenarios"
f4_3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_3 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

projection,year,scenario,high,historical,low,median,optimistic
0,1990,historical,NaN,35.989277,NaN,NaN,NaN
1,1991,historical,NaN,36.352906,NaN,NaN,NaN
2,1992,historical,NaN,35.529755,NaN,NaN,NaN
3,1993,historical,NaN,35.736477,NaN,NaN,NaN
4,1994,historical,NaN,35.857245,NaN,NaN,NaN


In [3191]:
base = alt.Chart(f4_3).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line1 = (
    base.mark_line(color=colors["eco-gray"])
    .encode(
        y=alt.Y(
            "historical:Q",
            axis=alt.Axis(
                grid=True,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".0f",
            ),
        )
    )
    .transform_filter('datum.scenario=="historical"')
)
line2 = (
    base.mark_line(color=colors["eco-mid-blue"])
    .encode(y=alt.Y("optimistic:Q"))
    .transform_filter('datum.scenario=="optimistic"')
)
line3 = (
    base.mark_line(color=colors["eco-turquiose"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c1"')
)
line4 = (
    base.mark_line(color=colors["eco-light-blue"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c2"')
)
area1 = line3.mark_area(color=colors["eco-turquiose"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area2 = line4.mark_area(color=colors["eco-light-blue"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area3 = (
    base.mark_area(color=colors["eco-mid-blue"], opacity=0.6)
    .encode(y="low:Q", y2="high:Q")
    .transform_filter('datum.scenario=="pledges"')
)
area4 = (
    base.mark_area(color=colors["eco-gray"], opacity=0.6)
    .encode(
        y="low:Q",
        y2="high:Q",
    )
    .transform_filter('datum.scenario=="current"')
)
ruler = (
    base.mark_line(color=colors["eco-gray"]).encode(y="y:Q").transform_calculate(y="0")
)
r2021 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 1.5),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2021")
)
text2021 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": 0, "t": "2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.5),
        align="left",
        baseline="bottom",
        dx=5,
        dy=-5,
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 43, "t": "Current policies"},
                {"x": 2100, "y": 40, "t": "2.7 - 3.1 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 21, "t": "Pledges & targets"},
                {"x": 2100, "y": 18, "t": "2.4 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 14, "t": "Net-zero targets"},
                {"x": 2100, "y": 11, "t": "2.0 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text4 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 2, "t": "<2 °C consistent"},
                {"x": 2100, "y": -1, "t": "1.6 - 1.7 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text5 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": -5, "t": "<1.5 °C consistent"},
                {"x": 2100, "y": -8, "t": "1.3 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Global greenhouse gas emissions and warming projections until 2100",
    subtitle=[
        "Global CO₂ emissions. Source: Climate Action Tracker, May 2021 briefing"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + r2021
            + area1
            + area2
            + area3
            + area4
            + line1
            + line2
            + line3
            + line4
            + text2021
            + text1
            + text2
            + text3
            + text4
            + text5
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3192]:
base = alt.Chart(f4_3).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line1 = (
    base.mark_line(color=colors["eco-gray"])
    .encode(
        y=alt.Y(
            "historical:Q",
            axis=alt.Axis(
                grid=True,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=22,
                titleY=-3,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".0f",
            ),
        )
    )
    .transform_filter('datum.scenario=="historical"')
)
line2 = (
    base.mark_line(color=colors["eco-mid-blue"])
    .encode(y=alt.Y("optimistic:Q"))
    .transform_filter('datum.scenario=="optimistic"')
)
line3 = (
    base.mark_line(color=colors["eco-turquiose"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c1"')
)
line4 = (
    base.mark_line(color=colors["eco-light-blue"], strokeDash=[5, 5], strokeWidth=1)
    .encode(y=alt.Y("median:Q"))
    .transform_filter('datum.scenario=="c2"')
)
area1 = line3.mark_area(color=colors["eco-turquiose"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area2 = line4.mark_area(color=colors["eco-light-blue"], opacity=0.6).encode(
    y="low:Q", y2="high:Q"
)
area3 = (
    base.mark_area(color=colors["eco-mid-blue"], opacity=0.6)
    .encode(y="low:Q", y2="high:Q")
    .transform_filter('datum.scenario=="pledges"')
)
area4 = (
    base.mark_area(color=colors["eco-gray"], opacity=0.6)
    .encode(
        y="low:Q",
        y2="high:Q",
    )
    .transform_filter('datum.scenario=="current"')
)
ruler = (
    base.mark_line(color=colors["eco-gray"]).encode(y="y:Q").transform_calculate(y="0")
)
r2021 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 0.7),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2021")
)
text2021 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": 0, "t": "2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 0.7),
        align="left",
        baseline="bottom",
        dx=5,
        dy=-5,
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 43, "t": "Current policies"},
                {"x": 2100, "y": 40, "t": "2.7 - 3.1 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 21, "t": "Pledges & targets"},
                {"x": 2100, "y": 18, "t": "2.4 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 14, "t": "Net-zero targets"},
                {"x": 2100, "y": 11, "t": "2.0 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-mid-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text4 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": 2, "t": "<2 °C consistent"},
                {"x": 2100, "y": -1, "t": "1.6 - 1.7 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text5 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2100, "y": -5, "t": "<1.5 °C consistent"},
                {"x": 2100, "y": -8, "t": "1.3 °C"},
            ]
        )
    )
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Global greenhouse gas emissions and warming projections until 2100",
    subtitle=[
        "Global CO₂ emissions. Source: Climate Action Tracker, May 2021 breifing"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-10,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + r2021
            + area1
            + area2
            + area3
            + area4
            + line1
            + line2
            + line3
            + line4
            + text2021
            + text1
            + text2
            + text3
            + text4
            + text5
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 4-4

In [3193]:
configSource = "raw/ECO2.DATA.C4.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"])

In [3194]:
f = "fig4-4_gdp-loss"
f4_4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_4 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Country,Scenario,Value 1
0,Thailand,1.4,-1.2
1,Phillipines,1.4,-1.3
2,Malaysia,1.4,-1.2
3,Saudi Arabia,1.4,-0.9
4,Indonesia,1.4,-0.6


In [3195]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_4
config["encoding"]["x"]["axis"]["grid"] = False
config["title"][
    "subtitle"
] = "% change in GDP, ordered by most affected country. Source: Swiss Re"
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"] = None
config["encoding"]["color"]["legend"]["labelColor"] = colors["eco-gray"]
config["encoding"]["color"]["legend"]["titleColor"] = colors["eco-gray"]
config["encoding"]["column"]["header"] = {}
config["encoding"]["column"]["header"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["encoding"]["y"]["axis"]["format"] = ".0%"
config["encoding"]["y"]["scale"] = {"domain": [-0.13, 0]}
config["encoding"]["y"]["field"] = "m"
config["title"]["dy"] = -5
config.pop("$schema")
config.pop("config")
config["view"] = {"stroke": None}
config["mark"] = {"type": "bar", "opacity": 0.9}
config["width"] = 50
config["height"] = 220
base = alt.Chart.from_dict(config).transform_calculate(m='datum["Value 1"]/100')
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
)
layer1 = (base).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.Chart(...)

In [3196]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

### Fig 4-5

In [3197]:
configSource = "raw/ECO2.DATA.C5.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"])

In [3198]:
f = "fig4-5_forest-area"
f4_5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_5 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Country,Value 1,Value 2
0,1990,Brazil,70.458021,0.0
1,1991,Brazil,70.005654,-0.6
2,1992,Brazil,69.553288,-1.3
3,1993,Brazil,69.100922,-1.9
4,1994,Brazil,68.648556,-2.6


In [3199]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_5
config["title"]["text"] = "Forest area by country"
config["title"][
    "subtitle"
] = "% change, in square km (Base year = 1990). Source: World Bank"
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["format"] = ".0%"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["encoding"]["y"]["field"] = "m"
config["encoding"]["y"]["scale"] = {"domain": [-0.4, 0.05]}
config["title"]["dy"] = -5
config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config).transform_calculate(m='datum["Value 2"]/100')
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2018")
)
layer1 = (base + text).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [3200]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 4-6

In [3201]:
configSource = "raw/ECO2.DATA.C6.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"])

In [3202]:
f = "fig4-6_endangered-species"
f4_6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4_6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4_6 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Type,Value,Category,Detail
0,Mammals,1323,Vertebrate,Value = Number of threatened species by 2021 (...
1,Birds,1481,Vertebrate,NaN
2,Reptiles,1458,Vertebrate,NaN
3,Amphibians,2442,Vertebrate,NaN
4,Fishes,3210,Vertebrate,NaN


In [3203]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f4_6
config["encoding"]["x"]["axis"] = {}
config["encoding"]["x"]["axis"]["grid"] = True
config["encoding"]["x"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleFontSize"] = 10
config["encoding"]["x"]["axis"]["titleAlign"] = "right"
config["encoding"]["x"]["axis"]["titleAnchor"] = "end"
config["encoding"]["x"]["axis"]["titleY"] = -15
config["encoding"]["x"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"] = {}
config["encoding"]["y"]["axis"]["grid"] = False
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"].pop("sort")
config["title"]["dy"] = -5
# config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["mark"] = {"type": "bar", "opacity": 0.9}
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config)
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2018")
)
layer1 = (base + text).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

WARN Infinite extent for field "sum_Value": [Infinity, -Infinity]
WARN Infinite extent for field "sum_Value": [Infinity, -Infinity]


alt.LayerChart(...)

In [3204]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

WARN Infinite extent for field "sum_Value": [Infinity, -Infinity]
WARN Infinite extent for field "sum_Value": [Infinity, -Infinity]


alt.LayerChart(...)

# Article 5

## Targets

In [3205]:
readme = "# Article 5 \n\n## Targets\n\n"
open("README.md", "a").write(readme)

26

In [3206]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/is-the-uk-on-track-to-meet-its-climate-commitments/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

200

### Fig 5-1

In [3207]:
f = "fig1_projected-emissions"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3208]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 5-2

In [3209]:
f = "fig2_patents"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3210]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

# Article 6

## Weather

In [3211]:
readme = "# Article 6 \n\n## Weather\n\n"
open("README.md", "a").write(readme)

26

In [3212]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/what-are-the-economic-effects-of-extreme-weather-caused-by-climate-change/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

246

### Fig 6-1a

In [3213]:
f = "fig1a_extreme-events"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3214]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 6-1b

In [3215]:
f = "fig1b_extreme-event-dotplot"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3216]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

# Article 7

## Plastics

In [3217]:
readme = "# Article 7 \n\n## Plastics\n\n"
open("README.md", "a").write(readme)

27

### Fig 7-1

In [3218]:
configSource = "raw/ECO2.PLASTIC_C1.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3219]:
f = "fig7-1_ocean"
f7_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Scenario,Value
0,1950,Plastic waste growth up to 2050,0
1,1951,Plastic waste growth up to 2050,0
2,1952,Plastic waste growth up to 2050,0
3,1953,Plastic waste growth up to 2050,0
4,1954,Plastic waste growth up to 2050,100


In [3220]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f7_1
config["encoding"]["x"]["axis"] = {}
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleFontSize"] = 10
config["encoding"]["x"]["axis"]["titleAlign"] = "right"
config["encoding"]["x"]["axis"]["titleAnchor"] = "end"
config["encoding"]["x"]["axis"]["titleY"] = -15
config["encoding"]["x"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = "million tonnes"
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 25
config["encoding"]["y"]["axis"]["titleY"] = 19
config["encoding"]["y"].pop("sort")
config["encoding"]["y"]["field"] = "m"
config["title"]["subtitle"] = config["title"]["subtitle"].replace("Tonnes. ", "")
config["encoding"]["color"]["legend"]["offset"] = 40
config["encoding"]["color"]["legend"]["labelColor"] = colors["eco-gray"]
config["title"]["dy"] = -5
config.pop("$schema")
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config).transform_calculate(m="datum.Value/1000000")
layer1 = (base).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.Chart(...)

In [3221]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
base.encoding.color.legend.labelFontSize = 11
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

### Fig 7-2

In [3222]:
configSource = "raw/ECO2.PLASTIC.C2.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3223]:
f = "fig7-2_river"
f7_2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7_2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7_2 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,River,Value
0,"Amazon (Brazil, Peru, Colombia, Ecuador)",38900
1,Brantas (Indonesia),38900
2,"Cross (Nigeria, Cameroon)",40300
3,Dong (China),19100
4,"Ganges (India, Bangladesh)",115000


In [3224]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f7_2
config["encoding"]["x"]["axis"] = {}
config["encoding"]["x"]["axis"]["grid"] = True
config["encoding"]["x"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleFontSize"] = 10
config["encoding"]["x"]["axis"]["titleAlign"] = "right"
config["encoding"]["x"]["axis"]["titleAnchor"] = "end"
config["encoding"]["x"]["axis"]["titleY"] = -15
config["encoding"]["x"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"] = {}
config["encoding"]["y"]["axis"]["grid"] = False
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["labelLimit"] = 190
config["title"]["subtitle"] = config["title"]["subtitle"].replace(
    "Tonnes", "Thousand tonnes"
)
config["encoding"]["y"].pop("sort")
config["encoding"]["x"]["field"] = "m"
config["title"]["dy"] = -5
config.pop("$schema")
# config["encoding"]["color"]["legend"] = None
config["mark"] = {"type": "bar", "opacity": 0.9, "color": colors["eco-turquiose"]}
config["view"] = {"stroke": None}
config["width"] = 200
config["height"] = 330
base = alt.Chart.from_dict(config).transform_calculate(m="datum.Value/1000")
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2018")
)
layer1 = (base + text).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

WARN Infinite extent for field "m": [Infinity, -Infinity]
WARN Infinite extent for field "m": [Infinity, -Infinity]


alt.LayerChart(...)

In [3225]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Article 8

## JG interview

In [3226]:
readme = "# Article 8 \n\n## JG interview\n\n"
open("README.md", "a").write(readme)

31

### Fig 8-1

In [3227]:
configSource = "raw/ECO2.JG.C1c.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3228]:
f = "fig8-1_east-africa-gdp"
f8_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f8_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f8_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Country,Value
0,1960,Central Afican Republic,599.700618
1,1961,Central Afican Republic,619.348109
2,1962,Central Afican Republic,586.413076
3,1963,Central Afican Republic,572.140833
4,1964,Central Afican Republic,573.425922


In [3229]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f8_1
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = "$ / year / person"
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 35
config["encoding"]["y"]["axis"]["titleY"] = 12
config["title"]["dy"] = -5
config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config).transform_filter("datum.Country!='Tanzania'")
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2020")
    .transform_filter("datum.Country!='Central Afican Republic'")
    .transform_filter("datum.Country!='Democratic Republic of Congo'")
)
text2 = (
    base.mark_text(dx=5, align="left", text="Central African Republic")
    .transform_filter("year(datum.Year)==2012")
    .transform_filter("datum.Country=='Central Afican Republic'")
)
text3 = (
    base.mark_text(dx=5, align="left", text="DRC")
    .transform_filter("year(datum.Year)==2020")
    .transform_filter("datum.Country=='Democratic Republic of Congo'")
)
base2 = (
    alt.Chart.from_dict(config)
    .transform_filter("datum.Country=='Tanzania'")
    .transform_filter("year(datum.Year)>1987")
)
text4 = base2.mark_text(dx=5, align="left", text="Tanzania").transform_filter(
    "year(datum.Year)==2020"
)
layer1 = (base + base2 + text + text2 + text3 + text4).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [3230]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 1
base.encoding.y.axis.titleX += 5
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + base2 + text + text2 + text3 + text4).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

WARN Conflicting axis property "titleX" (40 and 35). Using 40.
WARN Conflicting axis property "titleY" (13 and 12). Using 13.
WARN Conflicting axis property "titleX" (40 and 35). Using 40.
WARN Conflicting axis property "titleY" (13 and 12). Using 13.
WARN Conflicting axis property "titleX" (40 and 35). Using 40.
WARN Conflicting axis property "titleY" (13 and 12). Using 13.
WARN Conflicting axis property "titleX" (40 and 35). Using 40.
WARN Conflicting axis property "titleY" (13 and 12). Using 13.


alt.LayerChart(...)

### Fig 8-2

In [3231]:
f = "fig8-2/map"
readme = (
    "### "
    + f
    + " --- [html]("
    + "visualisation/"
    + f.replace("map", "")
    + 'index.html "'
    + f
    + '") [svg]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)

275

In [3232]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [html]("
    + "visualisation/"
    + f.replace("map_dark", "")
    + 'index_dark.html "'
    + f
    + '") [svg]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)

335

# Article 9

## Transition

In [3233]:
readme = "# Article 9 \n\n## Transition\n\n"
open("README.md", "a").write(readme)

29

In [3234]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/what-are-the-likely-costs-of-the-transition-to-a-sustainable-economy/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

236

### Fig 9-1

In [3235]:
f = "fig1_lcoe"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3236]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 9-2

In [3237]:
f = "fig2_energy-investment"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3238]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 9-3

In [3239]:
f = "fig3_investment"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3240]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

# Article 11

## Post-growth

In [3241]:
readme = "# Article 11 \n\n## Post-growth\n\n"
open("README.md", "a").write(readme)

31

### Fig 11-1

In [3242]:
# https://data.worldbank.org/indicator/EN.ATM.CO2E.KD.GD
df = (
    pd.read_csv("raw/API_EN.ATM.CO2E.KD.GD_DS2_en_csv_v2_3016406.csv", skiprows=3)
    .drop(["Indicator Name", "Indicator Code", "Country Code", "Unnamed: 65"], axis=1)
    .set_index("Country Name")
    .stack()
    .reset_index()
)
df.columns = ["country", "year", "value"]

In [3243]:
f = "fig11-1_gdp-emissions-intensity"
f11_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f11_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f11_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,country,year,value
0,Aruba,1986,0.183050
1,Aruba,1987,0.392628
2,Aruba,1988,0.452976
3,Aruba,1989,0.428165
4,Africa Eastern and Southern,1960,0.767570


In [3244]:
base = alt.Chart(f11_1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            axis=alt.Axis(
                grid=True,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="kgCO₂e per 2010 US$ of GDP",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".1f",
            ),
        ),
        color=alt.Color(
            "country:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                ]
            ),
        ),
    )
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="country", oneOf=["United States", "United Kingdom", "European Union"]
        )
    )
)
line = (
    base.mark_line(color=colors["eco-gray"], strokeWidth=3)
    .encode(y="value:Q")
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["World"]))
)
labels = (
    lines.mark_text(align="left", dx=5)
    .encode(
        text="country:N",
    )
    .transform_filter("datum.year==2018")
)
label = (
    line.mark_text(
        align="left", dx=5, fontSize=12, fontStyle="bold", color=colors["eco-gray"]
    )
    .encode(text="country:N")
    .transform_filter("datum.year==2018")
)
title = alt.TitleParams(
    "Evolution of the emissions intensity of GDP",
    subtitle=["Emissions intensity of GDP. Source: World Bank Development Indicators"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((lines + line + labels + label).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
with alt.data_transformers.disable_max_rows():
    layer1.display()

alt.LayerChart(...)

In [3245]:
base = alt.Chart(f11_1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            axis=alt.Axis(
                grid=True,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="kgCO₂e per 2010 US$ of GDP",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-2,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".1f",
            ),
        ),
        color=alt.Color(
            "country:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                ]
            ),
        ),
    )
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="country", oneOf=["United States", "United Kingdom", "European Union"]
        )
    )
)
line = (
    base.mark_line(color=colors["eco-gray"], strokeWidth=3)
    .encode(y="value:Q")
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["World"]))
)
labels = (
    lines.mark_text(align="left", dx=5)
    .encode(
        text="country:N",
    )
    .transform_filter("datum.year==2018")
)
label = (
    line.mark_text(
        align="left", dx=5, fontSize=12, fontStyle="bold", color=colors["eco-gray"]
    )
    .encode(text="country:N")
    .transform_filter("datum.year==2018")
)
title = alt.TitleParams(
    "Evolution of the emissions intensity of GDP",
    subtitle=["Emissions intensity of GDP. Source: World Bank Development Indicators"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((lines + line + labels + label).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
with alt.data_transformers.disable_max_rows():
    layer1.display()

alt.LayerChart(...)

# Article 12

## Centre page

In [3246]:
readme = "# Article 12 \n\n## Centre page\n\n"
open("README.md", "a").write(readme)

31

### Fig 12-1

In [3247]:
# https://www.datagraver.com/case/kyoto-cherry-blossom-full-flower-dates
# https://www1.ncdc.noaa.gov/pub/data/paleo/historical/phenology/japan/LatestVersion/
# https://n-kishou.com/corp/news-contents/sakura/?lang=en
df = pd.read_excel("raw/KyotoFullFlower7.xls", skiprows=25, usecols="A:C").dropna()
df.columns = ["year", "day", "date"]
df["date"] = "0" + df["date"].astype(str).str[0] + "-" + df["date"].astype(str).str[1:3]

In [3248]:
f = "fig12-1_cherry-blossom"
f12_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f12_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f12_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,year,day,date
11,812,92.0,04-01
14,815,105.0,04-15
30,831,96.0,04-06
50,851,108.0,04-18
52,853,104.0,04-14


In [3249]:
base = alt.Chart(f12_1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Year",
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
        scale=alt.Scale(domain=[800, 2021], nice=False),
    )
)
points = base.mark_point(
    color=colors["eco-dot"], fill=colors["eco-dot"], size=10, opacity=0.2
).encode(
    y=alt.Y(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="First full-flowering day",
            titleX=5,
            titleY=15,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=["03-15", "05-15"]),
    )
)
line = (
    points.mark_line(color=colors["eco-dot"])
    .encode(y=alt.Y("y:T", sort=[]))
    .transform_window(y="mean(date)", frame=[-25, 25])
)
line2 = (
    points.mark_line(color=colors["eco-gray"], opacity=0.5)
    .encode(y="y:T")
    .transform_window(y="mean(date)", frame=[-1000, 1000])
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 810, "y": "04-16", "t": "overall mean"}]))
    .mark_text(color=colors["eco-gray"], align="left", dy=2)
    .encode(x="x:Q", y="y:T", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 890, "y": "04-09", "t": "50-year moving window"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2021, "y": "04-14", "t": "14th"},
                {"x": 2021, "y": "04-17", "t": "April"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text4 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "04-05", "t": "5th"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
points2 = points.mark_point(fill=colors["eco-turquiose"]).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 1323, 1409])
)
points3 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26"}]))
    .mark_point(fill=colors["eco-turquiose"], opacity=0.8)
    .encode(x="x:Q", y="y:T")
)
text5 = (
    alt.Chart(pd.DataFrame([{"x": 1323, "y": "05-05", "t": "May 5th, 1323"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=-8
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text6 = (
    alt.Chart(pd.DataFrame([{"x": 1409, "y": "03-27", "t": "March 27th, 1409"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=15
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text7 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26", "t": "March 26th, 2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="right", dx=-7, dy=10
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
title = alt.TitleParams(
    "Full-flowering day of the cherry blossom in Kyoto, Japan",
    subtitle=[
        "Source: NOAA, based on Aono, Kazui (2008), Aono, Saito (2010) and Aono (2012)"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            points
            + line2
            + line
            + text1
            + text2
            + text3
            + text4
            + points2
            + points3
            + text5
            + text6
            + text7
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.


alt.LayerChart(...)

In [3250]:
base = alt.Chart(f12_1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Year",
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
        scale=alt.Scale(domain=[800, 2021], nice=False),
    )
)
points = base.mark_point(
    color=colors["eco-dot"], fill=colors["eco-dot"], size=10, opacity=0.2
).encode(
    y=alt.Y(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="First full-flowering day",
            titleX=5,
            titleY=15,
            titleBaseline="bottom",
            titleAngle=0,
            titleFontSize=12,
            labelFontSize=12,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=["03-15", "05-15"]),
    )
)
line = (
    points.mark_line(color=colors["eco-dot"])
    .encode(y=alt.Y("y:T", sort=[]))
    .transform_window(y="mean(date)", frame=[-25, 25])
)
line2 = (
    points.mark_line(color=colors["eco-gray"], opacity=0.5)
    .encode(y="y:T")
    .transform_window(y="mean(date)", frame=[-1000, 1000])
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 810, "y": "04-16", "t": "overall mean"}]))
    .mark_text(color=colors["eco-gray"], align="left", dy=2)
    .encode(x="x:Q", y="y:T", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 890, "y": "04-09", "t": "50-year moving window"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2021, "y": "04-14", "t": "14th"},
                {"x": 2021, "y": "04-17", "t": "April"},
            ]
        )
    )
    .mark_text(color=colors["eco-gray"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
text4 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "04-05", "t": "5th"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="x:Q", y="y:T", text="t:N")
)
points2 = points.mark_point(fill=colors["eco-turquiose"]).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 1323, 1409])
)
points3 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26"}]))
    .mark_point(fill=colors["eco-turquiose"], opacity=0.8)
    .encode(x="x:Q", y="y:T")
)
text5 = (
    alt.Chart(pd.DataFrame([{"x": 1323, "y": "05-05", "t": "May 5th, 1323"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=-8
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text6 = (
    alt.Chart(pd.DataFrame([{"x": 1409, "y": "03-27", "t": "March 27th, 1409"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="center", dy=15
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
text7 = (
    alt.Chart(pd.DataFrame([{"x": 2021, "y": "03-26", "t": "March 26th, 2021"}]))
    .mark_text(
        color=scale_lightness(colors["eco-turquiose"], 0.7), align="right", dx=-7, dy=10
    )
    .encode(x="x:Q", y="y:T", text="t:N")
)
title = alt.TitleParams(
    "Full-flowering day of the cherry blossom in Kyoto, Japan",
    subtitle=[
        "Source: NOAA, based on Aono, Kazui (2008), Aono, Saito (2010) and Aono (2012)"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            points
            + line2
            + line
            + text1
            + text2
            + text3
            + text4
            + points2
            + points3
            + text5
            + text6
            + text7
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.
WARN An ancestor parsed field "y" as number but a child wants to parse the field as date.


alt.LayerChart(...)

### Fig 12-2a

In [3251]:
# https://www.eea.europa.eu/data-and-maps/data/external/mass-balance-of-the-greenland
# https://www.epa.gov/climate-indicators/climate-change-indicators-glaciers
# https://wgms.ch/latest-glacier-mass-balance-data/
df = pd.read_csv("raw/glaciers_fig-1.csv", skiprows=6)
df.columns = ["year", "glaciers", "d"]
df = df.drop(["d"], axis=1)
df2 = pd.read_excel("raw/imbie_dataset_greenland_dynamics-2019_12_10.xlsx")
df2 = df2[
    [
        "Year",
        "Cumulative ice sheet mass change (Gt)",
        "Cumulative ice sheet mass change uncertainty (Gt)",
    ]
].dropna()
df2.columns = ["year", "greenland", "e"]
df2 = pd.concat([pd.DataFrame([{"year": 1991, "greenland": 0, "e": 0}]), df2])
df2["greenland1"] = df2["greenland"] - df2["e"] / 2
df2["greenland2"] = df2["greenland"] + df2["e"] / 2
df2 = df2.drop(["e"], axis=1)
df3 = pd.read_excel("raw/imbie_dataset-2018_07_23.xlsx")
df3 = df3[
    [
        "Year",
        "Cumulative ice mass change (Gt)",
        "Cumulative ice mass change uncertainty (Gt)",
    ]
]
df3.columns = ["year", "antarctica", "e"]
df3["antarctica1"] = df3["antarctica"] - df3["e"] / 2
df3["antarctica2"] = df3["antarctica"] + df3["e"] / 2
df3 = df3.drop(["e"], axis=1)
df0 = -df.set_index("year").join(df2.set_index("year")).join(df3.set_index("year"))
df = df0.drop("glaciers", axis=1).reset_index()

In [3252]:
f = "fig12-2a_ice"
f12_2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f12_2a.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f12_2a = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,year,greenland,greenland1,greenland2,antarctica,antarctica1,antarctica2
0,1956,NaN,NaN,NaN,NaN,NaN,NaN
1,1957,NaN,NaN,NaN,NaN,NaN,NaN
2,1958,NaN,NaN,NaN,NaN,NaN,NaN
3,1959,NaN,NaN,NaN,NaN,NaN,NaN
4,1960,NaN,NaN,NaN,NaN,NaN,NaN


In [3253]:
base = (
    alt.Chart(f12_2a)
    .encode(
        x=alt.X(
            "year:Q",
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                orient="bottom",
                labelAngle=0,
                format=".0f",
                # zindex=1,
                # offset=-43
            ),
            scale=alt.Scale(domain=[1991, 2020]),
        )
    )
    .transform_filter("datum.year<2018")
    .transform_filter("datum.year>1990")
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "greenland:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="Gt of ice",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=28,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 4300]),
    )
)
line2 = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y("antarctica:Q"))
area1 = line1.mark_area(color=colors["eco-turquiose"], opacity=0.5).encode(
    y="greenland1:Q", y2="greenland2:Q"
)
area2 = line2.mark_area(color=colors["eco-light-blue"], opacity=0.5).encode(
    y="antarctica1:Q", y2="antarctica2:Q"
)
ruler = (
    alt.Chart(pd.DataFrame([{"x": 1991, "y": 0}, {"x": 2020, "y": 0}]))
    .mark_line(color=colors["eco-gray"])
    .encode(y="y:Q", x="x:Q")
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 3800, "t": "Greenland"}]))
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 2600, "t": "Antarctica"}]))
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Cumulative ice mass loss of ice sheets",
    subtitle=["Source: IMBIE/EEA/ESA"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1 + area2 + line1 + line2 + text1 + text2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3254]:
base = (
    alt.Chart(f12_2a)
    .encode(
        x=alt.X(
            "year:Q",
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                titleFontSize=12,
                labelFontSize=12,
                orient="bottom",
                labelAngle=0,
                format=".0f",
                # zindex=1,
                # offset=-43
            ),
            scale=alt.Scale(domain=[1991, 2020]),
        )
    )
    .transform_filter("datum.year<2018")
    .transform_filter("datum.year>1990")
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "greenland:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="Gt of ice",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=40,
            titleY=31,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 4300]),
    )
)
line2 = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y("antarctica:Q"))
area1 = line1.mark_area(color=colors["eco-turquiose"], opacity=0.5).encode(
    y="greenland1:Q", y2="greenland2:Q"
)
area2 = line2.mark_area(color=colors["eco-light-blue"], opacity=0.5).encode(
    y="antarctica1:Q", y2="antarctica2:Q"
)
ruler = (
    alt.Chart(pd.DataFrame([{"x": 1991, "y": 0}, {"x": 2020, "y": 0}]))
    .mark_line(color=colors["eco-gray"])
    .encode(y="y:Q", x="x:Q")
)
text1 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 3800, "t": "Greenland"}]))
    .mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(pd.DataFrame([{"x": 2017, "y": 2600, "t": "Antarctica"}]))
    .mark_text(color=colors["eco-light-blue"], align="left", dx=5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Cumulative ice mass loss of ice sheets",
    subtitle=["Source: IMBIE/EEA/ESA"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1 + area2 + line1 + line2 + text1 + text2).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 12-2b

In [3255]:
df = df0["glaciers"].reset_index()

In [3256]:
f = "fig12-2b_glaciers"
f12_2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f12_2b.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f12_2b = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,year,glaciers
0,1956,-0.000
1,1957,0.120
2,1958,0.976
3,1959,1.414
4,1960,1.968


In [3257]:
base = alt.Chart(f12_2b).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-light-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "glaciers:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="m of water equivalent",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=27,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
title = alt.TitleParams(
    "Cumulative mass loss of glaciers worldwide",
    subtitle=["Source: EPA, based on WGMS"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

In [3258]:
base = alt.Chart(f12_2b).encode()
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-light-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-background"], offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            labelFontSize=12,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    ),
    y=alt.Y(
        "glaciers:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="m of water equivalent",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=23,
            titleY=29,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
title = alt.TitleParams(
    "Cumulative mass loss of glaciers worldwide",
    subtitle=["Source: EPA, based on WGMS"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (area)
    .properties(height=300, width=400, title=title)
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Article 13

## Promises

In [3259]:
readme = "# Article 13 \n\n## Promises\n\n"
open("README.md", "a").write(readme)

28

### Fig 13-1

In [3260]:
# Tiloka
df = pd.read_csv("raw/ECO2.TENREYO.C1.csv").drop("Detail", axis=1)

In [3261]:
f = "fig13-1_co2targets"
f13_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f13_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f13_1 = df

readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,country,CountryCode,Value1,Value2
0,Antigua and Barbuda,ATG,73.865440,-80.052870
1,Australia,AUS,0.544275,-0.601545
2,Austria,AUT,43.387310,5.149522
3,Belarus,BLR,-58.704910,3.913097
4,Belgium,BEL,26.009290,8.748268


In [3262]:
base = alt.Chart(f13_1).encode(
    x=alt.X(
        "Value1:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            titleAlign="right",
            titleAnchor="end",
            title="Target reduction in emissions from starting year (%)",
            titleFontSize=10,
            labelFontSize=10,
            titleY=-15,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[-150, 100]),
    )
)
points = base.mark_point(
    clip=True,
    color=colors["eco-turquiose"],
    fill=colors["eco-turquiose"],
    size=50,
).encode(
    y=alt.Y(
        "Value2:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="Actual reduction in emissions between 2010-2018 (%)",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            labelFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=5,
            titleY=7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[-62, 50], nice=False),
    )
)
# labels=bar.mark_text(color=colors['eco-turquiose'],align='right',dx=-5,baseline='middle').encode(text='Policy:N',x='x:Q').transform_calculate(x='min(0,datum.Average)')
title = alt.TitleParams(
    "Planned and actual reductions in emissions",
    subtitle=[
        "Negative values represent an emissions increase! Source: Tenreyo and de Silva (2021)"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
line = points.mark_line(clip=True, color=colors["eco-turquiose"], opacity=0.8).encode(
    y="Value1:Q"
)
zero1 = (
    line.mark_line(clip=True, color=colors["eco-gray"], opacity=0.5)
    .encode(x="z:Q")
    .transform_calculate(z="0")
)
zero2 = (
    line.mark_line(clip=True, color=colors["eco-gray"], opacity=0.5)
    .encode(y="z:Q")
    .transform_calculate(z="0")
)
labels = (
    points.mark_text(clip=True, color=colors["eco-gray"], dy=12)
    .encode(text="CountryCode:N")
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="CountryCode",
            oneOf=[
                "PNG",
                "IND",
                "IDN",
                "MDA",
                "UKR",
                "MLT",
                "GBR",
                "USA",
                "CHN",
                "ROU",
                "LTU",
                "CYP",
                "IRL",
                "JPN",
                "BRA",
                "RUS",
                "SGP",
                "NZL",
                "KOR",
                "CAN",
                "DNK",
                "FIN",
                "EUU",
                "ITA",
                "FRA",
                "DEU",
                "POL",
                "SVN",
                "FIN",
                "CAN",
            ],
        )
    )
)
layer1 = (
    ((zero1 + zero2 + line + points + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3263]:
points.encoding.x.axis.titleFontSize = 12
points.encoding.y.axis.titleFontSize = 12
# points.encoding.y.axis.titleY += 2

theme = "_dark"
layer1 = (
    ((zero1 + zero2 + line + points + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

#### Fig 13-2a

In [3264]:
configSource = "raw/ECO2.TENREYO.C2a.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3265]:
f = "fig13-2a_emissions-by-region"
f13_2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f13_2a.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f13_2a = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Region,Value1,Value2
0,1970,Sub Saharan Africa,271.4065,0.915174
1,1971,Sub Saharan Africa,284.8900,0.936775
2,1972,Sub Saharan Africa,307.0701,0.983175
3,1973,Sub Saharan Africa,350.0824,1.091057
4,1974,Sub Saharan Africa,379.0392,1.150727


In [3266]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f13_2a
config["layer"][0]["encoding"]["x"]["axis"]["format"] = ".0f"
config["layer"][0]["encoding"]["x"]["axis"]["grid"] = False
config["layer"][0]["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["grid"] = True
config["layer"][0]["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["layer"][0]["encoding"]["y"]["axis"]["title"] = "GtCO₂e / year"
config["title"][
    "subtitle"
] = "Gigatonnes of CO₂ equivalent. Source: Author's calculations"
config["layer"][0]["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["layer"][0]["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["layer"][0]["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["layer"][0]["encoding"]["y"]["axis"]["ticks"] = False
config["layer"][0]["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["layer"][0]["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["layer"][0]["encoding"]["y"]["axis"]["labelPadding"] = -5
config["layer"][0]["encoding"]["y"]["axis"]["labelOffset"] = -10
config["layer"][0]["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["layer"][0]["encoding"]["y"]["axis"]["titleAngle"] = 0
config["layer"][0]["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["layer"][0]["encoding"]["y"]["axis"]["tickCount"] = 7
config["layer"][0]["encoding"]["y"]["axis"]["titleX"] = 20
config["layer"][0]["encoding"]["y"]["axis"]["titleY"] = -5
config["layer"][0]["encoding"]["y"]["field"] = "m"
config["layer"][0]["encoding"]["x"]["type"] = "quantitative"
# config["layer"][1]["encoding"]["x"]["field"] = "p"
# config["layer"][1]["encoding"]["x"]["type"] = "quantitative"
# config["layer"][1]["data"].pop('format');
config["layer"][0]["encoding"]["color"]["legend"] = None
config["layer"][1]["mark"]["color"] = colors["eco-gray"]
config["layer"][1]["mark"]["opacity"] = 0.1
config["layer"]=[config["layer"][1],config["layer"][0]]
config['layer'][0]['encoding']['x']['scale']={'domain':[1970,2020]}
config["title"]["dy"] = -5
config.pop("$schema")
config.pop("autosize")
config["view"] = {"stroke": None}
config["width"] = 400
config["height"] = 300
base = (
    alt.Chart.from_dict(config)
    .transform_calculate(m="datum.Value1/1000")
    .transform_calculate(p="year(toDate(datum.Lines))")
)
text = (
    (
        alt.Chart(f13_2a)
        .mark_text(align="left", dx=5)
        .encode(
            x="Year:Q",
            y=alt.Y("m:Q"),
            text="Region:N",
            color="Region:N",
        )
    )
    .transform_calculate(m="datum.Value1/1000")
    .transform_filter("datum.Year==2018")
)
text1 = text.transform_filter("datum.Region!='South Asia'")
text2 = text.transform_filter("datum.Region=='South Asia'").mark_text(
    dy=-10, dx=5, align="left"
)
text3 = (
    alt.Chart(pd.DataFrame([{"y": 15.5, "t": "Kyoto", "x": 1997}, {"y": 14.8, "t": 1997, "x": 1997}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
text4 = (
    alt.Chart(pd.DataFrame([{"y": 15.5, "t": "Copenhagen", "x": 2009}, {"y": 14.8, "t": 2009, "x": 2009}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
text5 = (
    alt.Chart(pd.DataFrame([{"y": 11.5, "t": "Paris", "x": 2015}, {"y": 10.8, "t": 2015, "x": 2015}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
layer1 = (base + text1 + text2 + text3 + text4 + text5).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.LayerChart(...)

In [3267]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
text3.mark.color = scale_lightness(colors["eco-gray"], 0.9)
text4.mark.color = scale_lightness(colors["eco-gray"], 0.9)
text5.mark.color = scale_lightness(colors["eco-gray"], 0.9)
base.layer[1].encoding.y.axis.titleY += 1
base.layer[1].encoding.y.axis.titleX += 3
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text1 + text2 + text3 + text4 + text5).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.LayerChart(...)

#### Fig 13-2b

In [3268]:
configSource = "raw/ECO2.TENREYO.C2b.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3269]:
f = "fig13-2b_emissions-per-capita-by-region"
f13_2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f13_2b.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f13_2b = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Region,Value1,Value2
0,1970,Sub Saharan Africa,271.4065,0.915174
1,1971,Sub Saharan Africa,284.8900,0.936775
2,1972,Sub Saharan Africa,307.0701,0.983175
3,1973,Sub Saharan Africa,350.0824,1.091057
4,1974,Sub Saharan Africa,379.0392,1.150727


In [3270]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f13_2b
config["layer"][0]["encoding"]["x"]["axis"]["grid"] = False
config["layer"][0]["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["grid"] = True
config["layer"][0]["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["layer"][0]["encoding"]["y"]["axis"]["title"] = "tCO₂e / person / year"
config["title"]["subtitle"] = "Tonnes of CO₂ equivalent. Source: Author's calculations"
config["layer"][0]["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["layer"][0]["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["layer"][0]["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["layer"][0]["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["layer"][0]["encoding"]["y"]["axis"]["ticks"] = False
config["layer"][0]["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["layer"][0]["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["layer"][0]["encoding"]["y"]["axis"]["labelPadding"] = -5
config["layer"][0]["encoding"]["y"]["axis"]["labelOffset"] = -10
config["layer"][0]["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["layer"][0]["encoding"]["y"]["axis"]["titleAngle"] = 0
config["layer"][0]["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["layer"][0]["encoding"]["y"]["axis"]["tickCount"] = 7
config["layer"][0]["encoding"]["y"]["axis"]["titleX"] = 0
config["layer"][0]["encoding"]["y"]["axis"]["titleY"] = -5
config["layer"][0]["encoding"]["y"]["field"] = "m"
config["layer"][0]["encoding"]["x"]["type"] = "quantitative"
config["layer"][0]["encoding"]["x"]["axis"]['format'] = ".0f"
config["layer"][0]["encoding"]["color"]["legend"] = None
config["layer"][1]["mark"]["color"] = colors["eco-gray"]
config["layer"][1]["mark"]["opacity"] = 0.1
config["layer"] = [config["layer"][1], config["layer"][0]]
config['layer'][0]['encoding']['x']['scale']={'domain':[1970,2020]}
config["title"]["dy"] = -5
config.pop("$schema")
config.pop("autosize")
config["view"] = {"stroke": None}
config["width"] = 400
config["height"] = 300
base = alt.Chart.from_dict(config).transform_calculate(m="datum.Value2/1")
text = (
    (
        alt.Chart(f13_2b)
        .mark_text(align="left", dx=5)
        .encode(
            y=alt.Y("m:Q"),
            x='Year:Q',
            text="Region:N",
            color="Region:N",
        )
    )
    .transform_calculate(m="datum.Value2/1")
    .transform_filter("datum.Year==2018")
)
text1 = text.transform_filter("datum.Region!='Middle East and North Africa'")
text2 = text.transform_filter("datum.Region=='Middle East and North Africa'").mark_text(
    dy=6, dx=5, align="left"
)
text3 = (
    alt.Chart(pd.DataFrame([{"y": 23.5, "t": "Kyoto","x": 1997}, {"y": 22.5, "t": "1997","x": 1997}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
text4 = (
    alt.Chart(pd.DataFrame([{"y": 23.5, "t": "Copenhagen","x": 2009}, {"y": 22.5, "t": "2009","x": 2009}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
text5 = (
    alt.Chart(pd.DataFrame([{"y": 23.5, "t": "Paris","x": 2015}, {"y": 22.5, "t": "2015","x": 2015}]))
    .mark_text(
        color=scale_lightness(colors["eco-gray"], 1.4),
        align="right",
        dx=-5,
        fontSize=10,
    )
    .encode(y="y:Q", text="t:N",x='x:Q')
)
layer1 = (base + text1 + text2 + text3 + text4 + text5).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.LayerChart(...)

In [3271]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
text3.mark.color = scale_lightness(colors["eco-gray"], 0.9)
text4.mark.color = scale_lightness(colors["eco-gray"], 0.9)
text5.mark.color = scale_lightness(colors["eco-gray"], 0.9)
base.layer[1].encoding.y.axis.titleY += 1
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text1 + text2 + text3 + text4 + text5).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.LayerChart(...)

# Article 17

## Historic pollution

In [3272]:
readme = "# Article 17 \n\n## Historic pollution\n\n"
open("README.md", "a").write(readme)

38

In [3273]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/what-are-the-long-term-economic-effects-of-air-pollution/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

212

### Fig 17-1

In [3274]:
f = "fig1_coal"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3275]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 17-2a

In [3276]:
# Ordnance survey map
readme = "### Fig 17-2 Ordnance survey map\n\n"
open("README.md", "a").write(readme)

34

### Fig 17-3

In [3277]:
f = "fig3_pollution"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3278]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

# Article 18

## Productivity

In [3279]:
readme = "# Article 18 \n\n## Productivity\n\n"
open("README.md", "a").write(readme)

32

### Fig 18-1a

In [3280]:
# climate and productivity
# appendix table A1, available in the online version of the article.
# This should have the underlying results for fig 3a (high risk) and fig 4a (low risk)
df = pd.read_excel("raw/10694apa.xlsx")
df = df.dropna(axis=1, how="all")
for c in df.columns:
    df[c] = (
        df[c]
        .astype(str)
        .str.replace("—", "-")
        .str.replace(" ", "")
        .str.replace(" ", "")
    )
df.columns = ["temp", "l1", "o1", "l2", "o2", "l3", "o3", "o4"]
df = df.set_index("temp")
order = list(df.index[::2])[:-1]
df["columns"] = [i + j * "s" for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns1"] = [j * "s" for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns2"] = [i for i in list(df.index[::2]) for j in range(2)][:-1]
df["columns1"] = df["columns1"].replace("", "a")
df = (
    df.set_index(["columns2", "columns1"]).drop("columns", axis=1).stack().reset_index()
)
df.columns = ["temp", "cat", "measure", "value"]
df = df.set_index(["temp", "measure", "cat"]).unstack().dropna()["value"]
df["s"] = abs(df["s"].astype(float))
df["a"] = df["a"].astype(float)
df["a1"] = df["a"] - 2 * df["s"]
df["a2"] = df["a"] + 2 * df["s"]
df = (
    df.stack()
    .reset_index()
    .set_index(["temp", "measure", "cat"])
    .unstack()[0]
    .reset_index()
)
df = df.set_index("temp").loc[order].reset_index()

In [3281]:
f = "fig18-1a_time-allocation"
f18_1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f18_1a.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f18_1a = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

cat,temp,measure,a,a1,a2,s
0,<30,l1,6.423,-13.277,26.123,9.850
1,<30,l2,14.653,-37.431,66.737,26.042
2,<30,l3,1.324,-32.212,34.860,16.768
3,<30,o1,-37.132,-46.240,-28.024,4.554
4,<30,o2,-47.095,-77.729,-16.461,15.317


In [3282]:
base = alt.Chart(f18_1a).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=10,
            titleY=-15,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-mid-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=True,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-120, 80]),
        )
    )
    .transform_filter('datum.measure=="l2"')
)
area = line.mark_area(
    color=colors["eco-mid-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o2"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-mid-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-105")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["High-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + area
            + area2
            + line
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
            + label6
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3283]:
base = alt.Chart(f18_1a).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=12,
            labelFontSize=12,
            titleY=-17,
            titleX=420,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-light-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=False,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=42,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-120, 80]),
        )
    )
    .transform_filter('datum.measure=="l2"')
)
area = line.mark_area(
    color=colors["eco-light-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o2"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-light-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-102")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["High-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + area
            + area2
            + line
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
            + label6
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

### Fig 18-1b

In [3284]:
f = "fig18-1b_time-allocation-b"
f18_1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f18_1b.replace("/data/", "/visualisation/").replace(".csv", ".json"),
    )
)
if LOCAL:
    f18_1b = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

cat,temp,measure,a,a1,a2,s
0,<30,l1,6.423,-13.277,26.123,9.850
1,<30,l2,14.653,-37.431,66.737,26.042
2,<30,l3,1.324,-32.212,34.860,16.768
3,<30,o1,-37.132,-46.240,-28.024,4.554
4,<30,o2,-47.095,-77.729,-16.461,15.317


In [3285]:
base = alt.Chart(f18_1b).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=10,
            titleY=-15,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-mid-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=True,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-120, 80]),
        )
    )
    .transform_filter('datum.measure=="l3"')
)
area = line.mark_area(
    color=colors["eco-mid-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o3"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-mid-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-105")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=10
    )
    .encode(y="y:Q")
    .transform_calculate(y="-115")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["Low-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + area
            + area2
            + line
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
            + label6
        ).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3286]:
base = alt.Chart(f18_1b).encode(
    x=alt.X(
        "temp:O",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleFontSize=12,
            labelFontSize=12,
            titleY=-17,
            titleX=420,
            titleFontWeight="normal",
            title="maximum temperature (°F)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-light-blue"], interpolate="basis")
    .encode(
        y=alt.Y(
            "a:Q",
            axis=alt.Axis(
                grid=False,
                title="minutes",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=42,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-120, 80]),
        )
    )
    .transform_filter('datum.measure=="l3"')
)
area = line.mark_area(
    color=colors["eco-light-blue"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
line2 = (
    base.mark_line(color=colors["eco-turquiose"], interpolate="basis")
    .encode(y="a:Q")
    .transform_filter('datum.measure=="o3"')
)
area2 = line2.mark_area(
    color=colors["eco-turquiose"], interpolate="basis", opacity=0.2
).encode(y="a1:Q", y2="a2:Q")
ruler = (
    base.mark_line(color=colors["eco-gray"], strokeDash=[5, 5], opacity=0.5)
    .encode(y="y:Q")
    .transform_calculate(y="0")
)
label1 = line.mark_text(
    color=colors["eco-light-blue"], text="Labour", align="left", dx=5
).transform_filter('datum.temp==">100"')
label2 = line2.mark_text(
    color=colors["eco-turquiose"], text="Outdoor", align="left", dx=5
).transform_filter('datum.temp==">100"')
label3 = (
    line.mark_text(
        color=colors["eco-gray"], text=">38 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-102")
    .transform_filter('datum.temp==">100"')
)
label4 = (
    line.mark_text(
        color=colors["eco-gray"], text="8-10 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="46-50"')
)
label5 = (
    line.mark_text(
        color=colors["eco-gray"], text="19-21 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="66-70"')
)
label6 = (
    line.mark_text(
        color=colors["eco-gray"], text="< -1 °C", align="center", dx=7, fontSize=12
    )
    .encode(y="y:Q")
    .transform_calculate(y="-112")
    .transform_filter('datum.temp=="<30"')
)
title = alt.TitleParams(
    "Temperature and time allocation",
    subtitle=["Low-risk industries. Source: Zivin and Neidell (2021)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (
            ruler
            + area
            + area2
            + line
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
            + label6
        ).properties(height=300, width=400)
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Article 19

## Coal

In [3287]:
readme = "# Article 19 \n\n## Coal\n\n"
open("README.md", "a").write(readme)

24

In [3288]:
readme = (
    "### Fig 19-1 through 4: several charts in the article, but no specs or data\n\n"
)
open("README.md", "a").write(readme)

77

# Article 21

## Inequality

In [3289]:
readme = "# Article 21 \n\n## Inequality\n\n"
open("README.md", "a").write(readme)

30

In [3290]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/what-are-the-implications-of-decarbonisation-for-inequality/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

218

### Fig 21-1

In [3291]:
f = "fig1_outcomes"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3292]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 21-2

In [3293]:
# polar chart
readme = "### Fig 21-2 radar chart\n\n"
open("README.md", "a").write(readme)

26

In [3294]:
f = "fig2_polar"
readme = (
    "### "
    + f
    + " --- [pdf]("
    + p
    + "visualisation/"
    + f
    + '.pdf "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)

1006

# Article 22

## Agriculture

In [3295]:
readme = "# Article 22 \n\n## Agriculture\n\n"
open("README.md", "a").write(readme)

31

In [3296]:
p = (
    eco_git_path.split("main")[0]
    + "main"
    + "/articles/how-are-agricultural-economies-affected-by-climate-change/"
)
readme = (
    "Cloned from ["
    + p.split("/")[-2]
    + "]("
    + p.replace("raw.githubusercontent", "www.github").replace("/main/", "/tree/main/")
    + ")\n\n"
)
open("README.md", "a").write(readme)

214

### Fig 22-1

In [3297]:
f = "fig1_productivity"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3298]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 22-2

In [3299]:
f = "fig2_yield"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3300]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 22-3

In [3301]:
f = "fig3_africa"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

In [3302]:
f += "_dark"
readme = (
    "### "
    + f
    + " --- [json]("
    + p
    + "visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)
altair_viewer.display(
    json.loads(requests.get(p + "visualisation/" + f + ".json").content), inline=True
)

### Fig 22-4a

In [3303]:
f = "fig4a_rcp26"
readme = (
    "### "
    + f
    + " --- [kmz]("
    + p
    + "visualisation/"
    + f
    + '.kmz "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)

812

### Fig 22-4b

In [3304]:
f = "fig4b_rcp45"
readme = (
    "### "
    + f
    + " --- [kmz]("
    + p
    + "visualisation/"
    + f
    + '.kmz "'
    + f
    + '") [png]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")]('
    + p
    + "visualisation/"
    + f
    + '.png "'
    + f
    + '")\n\n'
)
open("README.md", "a").write(readme)

812

# Article 23

## Firms, competition

In [3305]:
readme = "# Article 23 \n\n## Firms, competition\n\n"
open("README.md", "a").write(readme)

38

### Fig 23-1

In [3306]:
df = pd.read_csv("raw/Gillingham_and_Stock_Table2.csv")

In [3307]:
f = "fig23-1_policies"
f23_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f23_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f23_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Policy,Average,Min,Max
0,Behavioral Energy Efficiency,-190.00,NaN,NaN
1,Reforestation,5.50,1.00,10.00
2,Clean Power Plan,11.00,NaN,NaN
3,Methane Flaring Regulation,20.00,NaN,NaN
4,EU ETS,30.62,8.56,52.68


In [3308]:
base = alt.Chart(f23_1).encode(y=alt.Y("Policy:N", sort=[], axis=None))
bar = base.mark_bar(color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "Average:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            titleAlign="right",
            titleAnchor="end",
            title="2017 US$ / ton of CO₂",
            titleFontSize=10,
            labelFontSize=10,
            titleY=-41,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            # zindex=1,
            # offset=-43
        ),
        #         scale=alt.Scale(domain=[-120,80])
    )
)
labels = (
    bar.mark_text(
        color=colors["eco-turquiose"], align="right", dx=-5, baseline="middle"
    )
    .encode(text="Policy:N", x="x:Q")
    .transform_calculate(x="min(0,datum.Average)")
)
title = alt.TitleParams(
    "Cost and efficiency-based comparison of green policies",
    subtitle=[""],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((bar + labels).properties(height=350, width=300))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [3309]:
bar.encoding.x.axis.titleFontSize = 12
bar.encoding.x.axis.titleY -= 5

theme = "_dark"
layer1 = (
    ((bar + labels).properties(height=350, width=300))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
# layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Article 24

## Public debt

In [3310]:
readme = "# Article 24 \n\n## Public debt\n\n"
open("README.md", "a").write(readme)

31

### Fig 24-1

In [3311]:
configSource = "raw/ECO2.DEBT.C1e.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [3312]:
f = "fig24-1_debt"
f24_1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f24_1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f24_1 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Year,Country,Value1,Value2,Value3,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,2002,Canada,0.760149,0.750681,98.754443,NaN,1000.00,NaN
1,2003,Canada,0.895599,0.752476,84.019275,NaN,NaN,NaN
2,2004,Canada,1.026470,0.758597,73.903447,NaN,1.00,USD
3,2005,Canada,1.173510,0.793334,67.603497,NaN,0.79,CAN
4,2006,Canada,1.319360,0.826672,62.657031,NaN,1.15,EUR


In [3313]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f24_1
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["encoding"]["y"]["axis"]["format"] = ".0%"
config["encoding"]["y"]["scale"] = {"domain": [0, 3]}
config["encoding"]["y"]["field"] = "m"
config["title"]["dy"] = -5
config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config).transform_calculate(m="datum.Value3/100")
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2020")
    .transform_filter("datum.Country!='UK'")
    .transform_filter("datum.Country!='United States'")
)
text2 = (
    base.mark_text(dx=5, dy=10, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2020")
    .transform_filter("datum.Country=='UK'")
)
text3 = (
    base.mark_text(dx=5, dy=-9, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Year)==2020")
    .transform_filter("datum.Country=='United States'")
)
layer1 = (base + text + text2 + text3).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [3314]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (base + text + text2 + text3).configure_view(stroke=None)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)